### Custom datasets

In [8]:
#Importing PyTorch and setting up device agnostic code
import torch
from torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"

torch.__version__

'2.0.0'

In [2]:
device

'cuda'

### GET DATA !!!!

Food 101 starts 101 different classes of food
we will start with 3 classes of food and %10 of all the images

It is generally a good idea to start small then increase the scale when necessary 

In [4]:
import requests 
import zipfile
from pathlib import Path

#Setup a path to the data folder
data_path=Path("data/")
image_path=data_path / "pizza_steak_sushi"

#If the image folder doesn't exist , download it and prepare it
if image_path.is_dir():
    print(f"{image_path} directory already exists , skipping download")
else:
    print(f"{image_path} creating directory")
    image_path.mkdir(parents=True,exist_ok=True)

#Download pizza,steak and sushi
with open(data_path/"pizza_steak_sushi.zip","wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading data")
    f.write(request.content)

#Unzip the data
with zipfile.ZipFile(data_path/"pizza_steak_sushi.zip","r") as zip_ref:
    print("Unzipping data..")
    zip_ref.extractall(image_path)


data\pizza_steak_sushi directory already exists , skipping download
Unzipping data..


In [5]:
from helper_functions import walk_through_dir
walk_through_dir(image_path)

There are 2 directories and 0 images in 'data\pizza_steak_sushi'.
There are 3 directories and 0 images in 'data\pizza_steak_sushi\test'.
There are 0 directories and 25 images in 'data\pizza_steak_sushi\test\pizza'.
There are 0 directories and 19 images in 'data\pizza_steak_sushi\test\steak'.
There are 0 directories and 31 images in 'data\pizza_steak_sushi\test\sushi'.
There are 3 directories and 0 images in 'data\pizza_steak_sushi\train'.
There are 0 directories and 78 images in 'data\pizza_steak_sushi\train\pizza'.
There are 0 directories and 75 images in 'data\pizza_steak_sushi\train\steak'.
There are 0 directories and 72 images in 'data\pizza_steak_sushi\train\sushi'.


In [6]:
#Setup train data and test data
train_dir=image_path/"train"
test_dir=image_path/"test"
train_dir,test_dir

(WindowsPath('data/pizza_steak_sushi/train'),
 WindowsPath('data/pizza_steak_sushi/test'))

In [ ]:
random_idx=torch.randint()